In [ ]:
#正常数据&PGD数据
import torch
import torchvision
import torchvision.transforms as transforms
from art.estimators.classification import PyTorchClassifier
import pandas as pd
import numpy as np
import art
import torch.nn as nn
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# 定义数据预处理
transform = transforms.Compose([
    transforms.ToTensor(),
])

# 加载CIFAR-10数据集
trainset = torchvision.datasets.CIFAR10(root='./cifar10', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./cifar10', train=False, download=True, transform=transform)

# 合并训练集和测试集
combined_data = trainset.data
combined_labels = trainset.targets
combined_data = np.concatenate((combined_data, testset.data), axis=0)
combined_labels = np.concatenate((combined_labels, testset.targets), axis=0)

# 获取类别名称
class_names = trainset.classes

# 创建一个字典来存储每个类别的DataFrame
dataframes = {}

# 按标签分组数据
for label in range(len(class_names)):
    # 获取当前标签的索引
    indices = np.where(combined_labels == label)[0]
    
    # 提取对应的数据和标签
    label_data = combined_data[indices]
    label_labels = combined_labels[indices]
    
    # 创建DataFrame
    df = pd.DataFrame({
        'image': [img for img in label_data],
        'label': label_labels
    })
    
    # 用类别名称命名DataFrame
    dataframes[class_names[label]] = df
    
# 从合并后的CSV文件中读取数据并恢复attacked_dataframes
def load_attacked_dataframes_from_csv(filename='merged_pgd.csv'):
    # 读取CSV文件
    all_data = pd.read_csv(filename)
    
    # 将图像数据从字符串格式转换回NumPy数组
    all_data['image'] = all_data['image'].apply(lambda x: np.frombuffer(eval(x), dtype=np.uint8).reshape(32, 32, 3))
    
    # 按类别名称分组并创建字典
    attacked_dataframes = {}
    for class_name in all_data['class_name'].unique():
        class_data = all_data[all_data['class_name'] == class_name]
        df = class_data[['image', 'label']].reset_index(drop=True)
        attacked_dataframes[class_name] = df
    
    return attacked_dataframes
pgds = load_attacked_dataframes_from_csv("./adv/deepfools.csv")

In [ ]:
#模型
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
import json


import torch
import torchvision.models as models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 定义与保存时一致的模型结构
"""class ModifiedResNet18(torch.nn.Module):
    def __init__(self, num_classes=10):
        super(ModifiedResNet18, self).__init__()
        self.resnet18 = models.resnet18(pretrained=False)  # 不使用预训练权重，因为我们加载的是已经训练好的模型
        self.resnet18.conv1 = torch.nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.resnet18.maxpool = torch.nn.Identity()  # 移除最大池化层
        
        # 获取ResNet18的特征提取部分
        self.features = torch.nn.Sequential(*list(self.resnet18.children())[:-1])
        
        # 添加MLP和全连接层
        self.mlp = torch.nn.Sequential(
            torch.nn.Linear(512, 256),
            torch.nn.ReLU(inplace=True),
            torch.nn.Dropout(0.5),
            torch.nn.Linear(256, 128),
            torch.nn.ReLU(inplace=True),
            torch.nn.Dropout(0.5)
        )
        self.fc = torch.nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.mlp(x)
        x = self.fc(x)
        return x"""
class ModifiedResNet18(torch.nn.Module):
    def __init__(self, num_classes=10):
        super(ModifiedResNet18, self).__init__()
        self.resnet18 = models.resnet18(pretrained=False)  # 不使用预训练权重，因为我们加载的是已经训练好的模型
        self.resnet18.conv1 = torch.nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.resnet18.maxpool = torch.nn.Identity()  # 移除最大池化层
        
        # 获取ResNet18的特征提取部分
        self.features = torch.nn.Sequential(*list(self.resnet18.children())[:-1])
        
        # 添加MLP和全连接层
        self.mlp = torch.nn.Sequential(
            torch.nn.Linear(512, 256),
            torch.nn.ReLU(inplace=True),
            torch.nn.Dropout(0.5),
            torch.nn.Linear(256, 128),
            torch.nn.ReLU(inplace=True),
            torch.nn.Dropout(0.5)
        )
        self.fc = torch.nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.mlp(x)
        x = self.fc(x)
        return x
    
    def get_all_activations(self, x, hook):
        # 前向传播
        with torch.no_grad():
            self(x)
        
        # 处理激活值
        features = []
        for name in hook.activations:
            layer_act = hook.activations[name].squeeze()
            if len(layer_act.shape) == 3:  # 卷积层特征
                features.append(torch.mean(layer_act, dim=(1, 2)))  # 全局平均池化
            else:  # 全连接层特征
                features.append(layer_act.flatten())
        return torch.cat(features, dim=1)


# 初始化模型
model = ModifiedResNet18()

# 加载保存的模型权重
model_path = 'best_model.pth'  # 替换为你的模型文件路径
model.load_state_dict(torch.load(model_path))
model.eval()
model.to(device)

In [ ]:
#钩子函数
class ActivationHook:
    def __init__(self):
        self.activations = {}
        
    def get_hook(self, name):
        def hook(module, input, output):
            self.activations[name] = output.detach()
        return hook

# 初始化钩子管理器
hook = ActivationHook()

# 为模型的所有卷积层和全连接层注册钩子
for name, module in model.named_modules():
    if isinstance(module, (torch.nn.Conv2d, torch.nn.Linear)):
        module.register_forward_hook(hook.get_hook(name))

In [ ]:
#特征提取器
def extract_features(image, transform):
    # 应用数据增强
    img_tensor = transform(image).unsqueeze(0).to(device)
    
    # 前向传播获取激活
    with torch.no_grad():
        model(img_tensor)
    
    # 处理各层激活
    features = []
    for name in hook.activations:
        layer_act = hook.activations[name].squeeze()
        if len(layer_act.shape) == 3:  # 卷积层特征
            features.append(torch.mean(layer_act, dim=(1, 2)))  # 全局平均池化
        else:  # 全连接层特征
            features.append(layer_act.flatten())
    return torch.cat(features).cpu().numpy()

In [ ]:
#数据增强手段
import torchvision.transforms as transforms

# 定义多种数据增强方法
advanced_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomApply([
        transforms.ColorJitter(0.8, 0.8, 0.8, 0.2)
    ], p=0.8),
    transforms.RandomGrayscale(p=0.2),
    transforms.RandomHorizontalFlip(),
    transforms.RandomResizedCrop(32, scale=(0.6, 1.0)),
    transforms.GaussianBlur(3),
    transforms.ToTensor(),
])

In [ ]:
from tqdm import tqdm
import numpy as np
# 修改后的build_dataset函数，生成视图对
def build_supervised_pairs(data_dict, label_type, num_samples=6000):
    pairs = []
    
    for class_name in tqdm(data_dict.keys()):
        df = data_dict[class_name]
        # 创建索引数组
        available_indices = np.arange(len(df))
        # 随机选择索引（允许重复）
        selected_indices = np.random.choice(available_indices, num_samples, replace=True)
        
        for idx in selected_indices:
            sample = df.iloc[idx]
            img = sample['image']
            
            # 为每个样本生成4个视图
            views = [extract_features(img, advanced_transform) for _ in range(4)]
            
            # 添加同样本正对
            for i in range(4):
                for j in range(i+1, 4):
                    pairs.append((views[i], views[j], label_type))
            
            # 添加跨样本正对（从同一类中随机选2个其他样本）
            other_indices = np.random.choice(available_indices, 2, replace=True)
            for other_idx in other_indices:
                other_sample = df.iloc[other_idx]
                other_view = extract_features(other_sample['image'], advanced_transform)
                pairs.extend([
                    (views[0], other_view, label_type),
                    (views[1], other_view, label_type)
                ])
    
    return pairs
    
# 构建配对数据集
normal_pairs = build_supervised_pairs(dataframes, 0)
adv_pairs = build_supervised_pairs(pgds, 1)
all_pairs = normal_pairs + adv_pairs

import os
import pickle

def save_pairs(adv_pairs, normal_pairs, folder_name):
    # 创建文件夹
    os.makedirs(folder_name, exist_ok=True)
    
    # 保存adv_pairs
    with open(os.path.join(folder_name, 'adv_pairs.pkl'), 'wb') as f:
        pickle.dump(adv_pairs, f)
    
    # 保存normal_pairs
    with open(os.path.join(folder_name, 'normal_pairs.pkl'), 'wb') as f:
        pickle.dump(normal_pairs, f)
    

    print(f"Data saved to {folder_name} successfully.")


save_pairs(adv_pairs, normal_pairs, 'deepfoolpairs')

"""
def load_pairs(folder_name):
    # 加载adv_pairs
    with open(os.path.join(folder_name, 'adv_pairs.pkl'), 'rb') as f:
        adv_pairs = pickle.load(f)
    
    # 加载normal_pairs
    with open(os.path.join(folder_name, 'normal_pairs.pkl'), 'rb') as f:
        normal_pairs = pickle.load(f)
    
    # 加载all_pairs
    all_pairs = normal_pairs + adv_pairs
    print(f"Data loaded from {folder_name} successfully.")
    return adv_pairs, normal_pairs, all_pairs"""



In [ ]:
# 划分训练集、验证集、测试集
from sklearn.model_selection import train_test_split
train_pairs, test_pairs = train_test_split(all_pairs, test_size=0.3, random_state=42)

# 创建Dataset类
class ContrastivePairDataset(torch.utils.data.Dataset):
    def __init__(self, pairs):
        self.pairs = pairs
        
    def __len__(self):
        return len(self.pairs)
    
    def __getitem__(self, idx):
        view1, view2, label = self.pairs[idx]
        return (
            torch.FloatTensor(view1),
            torch.FloatTensor(view2),
            torch.tensor(label, dtype=torch.long)
        )

# 创建DataLoader
train_dataset = ContrastivePairDataset(train_pairs)

test_dataset = ContrastivePairDataset(test_pairs)

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, drop_last=True)

test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import accuracy_score, roc_auc_score
import csv


# 定义对比学习模型
class ContrastiveDetector(nn.Module):
    def __init__(self, input_dim=5194):
        super(ContrastiveDetector, self).__init__()
        # 共享权重的双塔编码器
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 2048),
            nn.BatchNorm1d(2048),
            nn.ReLU(),
            nn.Dropout(0.3),
            
            nn.Linear(2048, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.3),
            
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
        )
        
        # 对比头（相似度度量）
        self.projection = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128)
        )
        
        # 分类头
        self.classifier = nn.Sequential(
            nn.Linear(512*2, 256),  # 拼接特征
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.2),
            
            nn.Linear(256, 1)
        )

    def forward(self, x1, x2):
        # 编码器处理
        emb1 = self.encoder(x1)
        emb2 = self.encoder(x2)
        
        # 对比学习分支
        proj1 = self.projection(emb1)
        proj2 = self.projection(emb2)
        
        # 分类分支
        combined = torch.cat([emb1, emb2], dim=1)
        logits = self.classifier(combined)
        
        return proj1, proj2, logits.squeeze()

# 复合损失函数：对比损失 + 分类损失
class HybridLoss(nn.Module):
    def __init__(self, temp=0.1, alpha=0.7):
        super().__init__()
        self.temp = temp
        self.alpha = alpha
        self.ce_loss = nn.BCEWithLogitsLoss()
        
    def contrastive_loss(self, proj1, proj2):
        # 计算标准化嵌入
        proj1 = nn.functional.normalize(proj1, dim=1)
        proj2 = nn.functional.normalize(proj2, dim=1)
        
        # 计算相似度矩阵
        logits = (proj1 @ proj2.T) / self.temp
        labels = torch.arange(logits.size(0)).to(logits.device)
        
        loss1 = nn.functional.cross_entropy(logits, labels)
        loss2 = nn.functional.cross_entropy(logits.T, labels)
        return (loss1 + loss2) / 2
    
    # 修改HybridLoss的forward方法
    def forward(self, proj1, proj2, logits, targets):
        cont_loss = self.contrastive_loss(proj1, proj2)
        cls_loss = self.ce_loss(logits, targets.float().squeeze())  # 添加squeeze()
        return self.alpha*cont_loss + (1-self.alpha)*cls_loss

# 修改后的训练函数
# 修改训练函数（添加准确率记录）
def train_model(model, train_loader, test_loader, epochs=50, lr=1e-4):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    
    history = {
        'epoch': [],
        'train_loss': [], 'train_acc': [], 'train_auc': [],
        'test_loss': [], 'test_acc': [], 'test_auc': []
    }
    
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
    criterion = HybridLoss(temp=0.2, alpha=0.5)
    
    best_auc = 0
    for epoch in range(epochs):
        # 训练阶段
        model.train()
        train_loss = 0
        train_preds, train_probs, train_targets = [], [], []
        
        for x1, x2, labels in train_loader:
            x1, x2, labels = x1.to(device), x2.to(device), labels.float().to(device)
            
            optimizer.zero_grad()
            proj1, proj2, logits = model(x1, x2)
            
            loss = criterion(proj1, proj2, logits, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            probs = torch.sigmoid(logits)
            train_probs.append(probs.detach().cpu())
            train_preds.append((probs > 0.5).long().cpu())
            train_targets.append(labels.cpu())
        
        # 计算训练指标
        train_probs = torch.cat(train_probs)
        train_preds = torch.cat(train_preds)
        train_targets = torch.cat(train_targets)
        train_acc = accuracy_score(train_targets, train_preds)
        train_auc = roc_auc_score(train_targets, train_probs)
        avg_train_loss = train_loss / len(train_loader)
        
        # 测试集评估
        test_loss, test_acc, test_auc = evaluate_during_training(model, test_loader, criterion, device)
        
        # 记录历史
        history['epoch'].append(epoch+1)
        history['train_loss'].append(avg_train_loss)
        history['train_acc'].append(train_acc)
        history['train_auc'].append(train_auc)
        history['test_loss'].append(test_loss)
        history['test_acc'].append(test_acc)
        history['test_auc'].append(test_auc)
        
        # 保存最佳模型
        if test_auc > best_auc:
            best_auc = test_auc
            torch.save(model.state_dict(), "CL_deepbest_model.pth")
        
        # 打印进度
        print(f"Epoch {epoch+1}/{epochs}")
        print(f"Train Loss: {avg_train_loss:.4f} | Acc: {train_acc:.4f} | AUC: {train_auc:.4f}")
        print(f"Test  Loss: {test_loss:.4f} | Acc: {test_acc:.4f} | AUC: {test_auc:.4f}")
        print("--------------------------------------------------")
        
        # 保存记录到CSV
        with open('deepftraining_log.csv', 'a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([
                epoch + 1,
                avg_train_loss,
                train_acc,
                train_auc,
                test_loss,
                test_acc,
                test_auc
            ])

    
    return history

# 修改评估函数（返回准确率和AUC）
def evaluate_during_training(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    all_preds, all_probs, all_targets = [], [], []
    
    with torch.no_grad():
        for x1, x2, labels in loader:
            x1, x2, labels = x1.to(device), x2.to(device), labels.float().to(device)
            
            _, _, logits = model(x1, x2)
            loss = criterion.ce_loss(logits, labels)
            
            total_loss += loss.item()
            probs = torch.sigmoid(logits)
            all_probs.append(probs.cpu())
            all_preds.append((probs > 0.5).long().cpu())
            all_targets.append(labels.cpu())
    
    avg_loss = total_loss / len(loader)
    all_probs = torch.cat(all_probs)
    all_preds = torch.cat(all_preds)
    all_targets = torch.cat(all_targets)
    
    acc = accuracy_score(all_targets, all_preds)
    auc = roc_auc_score(all_targets, all_probs)
    return avg_loss, acc, auc





# 初始化并训练模型
model = ContrastiveDetector()
history = train_model(model, train_loader, test_loader, epochs=20, lr=2e-4)

